In [113]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

In [114]:
State = namedtuple('State', ['taken', 'not_taken'])


def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def will_cover(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.not_taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

In [115]:
PROBLEM_SIZE = 10
NUM_SETS = 15
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [122]:
#distanza dal trovare la soluzione dallo stato State
def h(state):
    #trovo il set che copre di più
    largest_set_size = max(sum(s) for s in SETS)
    #numero di pezzi che mancano da coprire
    missing_size = PROBLEM_SIZE - sum(covered(state))

    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    #vettore con la somma dei pezzi coperti [false, true, false, true...]
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    #trovo quanti pezzi non coperti ci sono
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


# i count the total number of blocks already covered and the total number of blocks that the not taken sets would cover 
# without considering that the not taken sets will cover blocks that are already covered. In general it solves the problem in less steps than h,
# but sometimes it doesen't work better than h2 and h3
def h4(state):
    already_covered = sum(covered(state)) 
    reverse_state = sum(will_cover(state))

    optimistic_estimate = ceil(reverse_state / (already_covered + 1))
    return optimistic_estimate

def f(state):
    return len(state.taken) + h2(state)

In [123]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

0it [00:00, ?it/s]

82it [00:00, 1232.93it/s]

Solved in 82 steps (4 tiles)
